### Задача: обучить нейронную сеть генерировать новые ингридиенты для зелий.

In [103]:
import numpy as np
import pandas as pd
import time
import torch

Загрузка данных

In [104]:
df = pd.read_csv('.../Potions.csv', sep=';')

In [73]:
df.head(10)

,Name,Known ingredients,Effect,Characteristics,Difficulty level
0,Ageing Potion,"Newt spleens , Bananas",Ages drinker temporarily,Green,Advanced
1,Amortentia,NaN,Love Potion that causes a powerful infatuation...,"Mother-of-pearl sheen, Spiralling steam, Scent...",Advanced
2,Antidote to Veritaserum,NaN,Counters the effect of Veritaserum,NaN,NaN
3,Babbling Beverage,"Valerian sprigs, Aconite, Dittany",Causes the drinker to speak nonsense,NaN,NaN
4,Baruffio's Brain Elixir,"Leaping Toadstools, Frog Brains, Runespoor egg...",Allegedly increases one's brain power,Green in colour,NaN
5,Befuddlement Draught,"Scurvy grass, Lovage, Sneezewort",Recklessness,Dark green in colour,NaN
6,First Love Beguiling Bubbles,NaN,Causes the drinker to become infatuated with t...,NaN,NaN
7,Fire Protection Potion,"Bursting mushrooms, Salamander blood, Wartcap ...",Protects drinker from fire,Purple or Black,Beginner
8,Tolipan Blemish Blitzer,Dragon claw,Treats acne,"White in colour, Thick paste consistency",NaN
9,Blood-Replenishing Potion,NaN,Replenishes lost blood,Dark red in colour,NaN


Предобработка данных

In [74]:
df.columns = [name.lower() for name in df.columns] # смена регистра в названии столбцов для удобства обращения

In [75]:
ingridients = df['known ingredients'].tolist() # преобразование датафрейма в список

In [76]:
ingridients = [i.split(',') for i in ingridients if isinstance(i, str)] # очищение от Nanи запятых

In [77]:
ingridients = [i.strip() for lst in ingridients for i in lst] # превращение в одни массив

In [78]:
ingridients = [i.lower() for i in ingridients] # приведение к нижнему регистру

In [79]:
ingridients = [[c for c in i if c.isalpha() or c.isspace()] for i in ingridients] # разбиение на буквы и пробелы

In [80]:
ingridients[:5]

[['n', 'e', 'w', 't', ' ', 's', 'p', 'l', 'e', 'e', 'n', 's'],
 ['b', 'a', 'n', 'a', 'n', 'a', 's'],
 ['v', 'a', 'l', 'e', 'r', 'i', 'a', 'n', ' ', 's', 'p', 'r', 'i', 'g', 's'],
 ['a', 'c', 'o', 'n', 'i', 't', 'e'],
 ['d', 'i', 't', 't', 'a', 'n', 'y']]

Подготовка данных для подачи в нейросеть (оцифровывание строчных символов)

In [81]:
chars = set('abcdefghijklmnopqrstuvwxyz ')
index_to_char = ['none'] + [w for w in chars]
char_to_index = {w: i for i, w in enumerate(index_to_char)}

In [82]:
char_to_index.items()

dict_items([('none', 0), ('c', 1), ('u', 2), ('w', 3), ('q', 4), ('r', 5), ('z', 6), ('o', 7), ('d', 8), ('s', 9), ('f', 10), ('g', 11), ('a', 12), ('p', 13), ('y', 14), ('i', 15), ('b', 16), ('l', 17), ('e', 18), ('x', 19), ('t', 20), ('h', 21), ('k', 22), ('n', 23), ('j', 24), ('m', 25), ('v', 26), (' ', 27)])

Перевод ингридиентов из строкового вида в числовой для подачи в нейросеть

In [83]:
# Определение максимально длинного названия ингридиента
# чтобы выбрать количество нейронов в первом слое нейросети
max_len = max([len(i) for i in ingridients])
max_len

57

In [84]:
X = torch.zeros((len(ingridients), max_len), dtype=int)

In [85]:
for i in range(len(ingridients)):
    for j, w in enumerate(ingridients[i]):
        if j >= max_len:
            break
        X[i, j] = char_to_index.get(w, char_to_index['none'])

In [86]:
X[:1]

tensor([[23, 18,  3, 20, 27,  9, 13, 17, 18, 18, 23,  9,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0]])

Определение нейросети

In [87]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(index_to_char), 28)
        self.gru = torch.nn.GRU(28, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(index_to_char))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [88]:
model = Network()

In [89]:
model.forward(X[:1])[0].shape

torch.Size([1, 57, 28])

In [90]:
def generate_ingridient(start_letter='a'):
    sentence = [start_letter]
    state = None
    for i in range(max_len):
        X = torch.Tensor([[char_to_index[sentence[i]]]]).type(torch.long)
        if i == 0:
            result, state = model.forward(X)
        else:
            result, state = model.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(index_to_char[index_of_prediction])
        
    print(''.join(sentence))

In [107]:
generate_ingridient('s')

sla bday


Обучение модели

In [93]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)

In [94]:
X_train = X[:, :-1]
Y_train = X[:, 1:].flatten()

In [96]:
for ep in range(200):
    start =time.time()
    train_loss = 0
    train_passed = 0
    
    optimizer.zero_grad()
    answers, _ = model.forward(X_train)
    answers = answers.view(-1, len(index_to_char))
    loss = criterion(answers, Y_train)
    train_loss += loss.item()
    
    loss.backward()
    optimizer.step()
    train_passed += 1
    
    print('Ep: {}. Time: {:.3f}. Train loss: {:.3f}'.format(ep, time.time() - start,
                                                            train_loss / train_passed))
    generate_ingridient()

Ep: 0. Time: 0.275. Train loss: 3.172
a
Ep: 1. Time: 0.238. Train loss: 5.533
a
Ep: 2. Time: 0.237. Train loss: 16.437
aocsc b
Ep: 3. Time: 0.249. Train loss: 7.011
auooogiaoooogiauuuugiauuuuuuuuuug ieloauooooooooooooooooao
Ep: 4. Time: 0.262. Train loss: 8.266
aooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
Ep: 5. Time: 0.258. Train loss: 8.513
admesooooooooooooooooooooooooooooooooooooooooooooooooooooo
Ep: 6. Time: 0.271. Train loss: 8.415
aooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
Ep: 7. Time: 0.270. Train loss: 8.729
aooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
Ep: 8. Time: 0.267. Train loss: 8.838
adiooooooooooooooooooooooooooooooooooooooooooooooooooooooo
Ep: 9. Time: 0.272. Train loss: 7.804
awe
Ep: 10. Time: 0.280. Train loss: 7.324
awe
Ep: 11. Time: 0.267. Train loss: 7.022
ar
Ep: 12. Time: 0.282. Train loss: 6.233
an blspr
Ep: 13. Time: 0.277. Train loss: 6.138
an basbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
Ep: 14. Time: 0.28

Ep: 131. Time: 0.391. Train loss: 1.177
at yotp
Ep: 132. Time: 0.404. Train loss: 1.176
ag mza ddddddddddddddddddddddddddddddddddddddddddddddddddd
Ep: 133. Time: 0.391. Train loss: 1.158
anakrakr
Ep: 134. Time: 0.385. Train loss: 1.152
anakrakr
Ep: 135. Time: 0.398. Train loss: 1.052
an waphpndywih
Ep: 136. Time: 0.397. Train loss: 1.056
a s s s s s s s s s sts
Ep: 137. Time: 0.391. Train loss: 1.000
ales
Ep: 138. Time: 0.394. Train loss: 0.968
anakrakr
Ep: 139. Time: 0.401. Train loss: 0.968
asptlontocpowsptohooneedmexwdentwddintcurat
Ep: 140. Time: 0.406. Train loss: 0.948
asprsywif wd f g ng ckent
Ep: 141. Time: 0.397. Train loss: 0.876
as on act fur age qh r egong caker
Ep: 142. Time: 0.388. Train loss: 0.861
andinersy
Ep: 143. Time: 0.400. Train loss: 0.856
ans
Ep: 144. Time: 0.401. Train loss: 0.838
agof orwopo jgom st  grass
Ep: 145. Time: 0.394. Train loss: 0.821
at  fa mamonomzy  fblsslsslslslslslslslslslslslslslslslsls
Ep: 146. Time: 0.390. Train loss: 0.793
at ye
Ep: 147. Ti

#### Вывод: в процессе запуска модели видно, что значение loss-функции снижается. Это может говорить о том, что модель обучается. Хотя модель не возвращает осознанных названий ингридиентов, в процессе можно видеть, что вначале модель периодически зацикливает возвращаемые символы, а после обучения можно видеть наборы символов более похожие на слова.